In [91]:
import pandas as pd, numpy as np, matplotlib.pyplot as plt, seaborn as sns
from sklearn.feature_extraction.text import CountVectorizer

from funciones_varias import cargaCsvToDataFrame

dfMoviesFinal = cargaCsvToDataFrame("dfMoviesFinal", "datasets_eda")

In [92]:
#considero que para el modelo de clasificacion no necesito estas columnas
dfMoviesFinal = dfMoviesFinal.drop(columns=["tagline", "id", "status", ""])

In [93]:
#dropeo estas rows porque considero que tienen generos de pelicula que no me sirven para el modelo, son datos erroneos

dfMoviesFinal.drop(dfMoviesFinal[dfMoviesFinal["movie_genre"]== "Aniplex"].index, inplace=True)
dfMoviesFinal.drop(dfMoviesFinal[dfMoviesFinal["movie_genre"]== "Odyssey Media"].index, inplace=True)
dfMoviesFinal.drop(dfMoviesFinal[dfMoviesFinal["movie_genre"]== "Carousel Productions"].index, inplace=True)

In [94]:
#considero que para poder continuar con el modelo de clasificacion, debo poner el tipo de dato que corresponde a cada columna

strings= ["movie_title", "movie_genre", "original_language", "overview", "production_company",
           "language", "franquicia", "production_country", "director", "character", "actor"]
integers=["release_year", "vote_count"]
floats=["runtime","return", "budget", "vote_average", "revenue", "popularity"]

#dropeo valor raro porque sino no puedo ponerle el data type que corresponde
dfMoviesFinal.drop(dfMoviesFinal[dfMoviesFinal['popularity'] == 'Beware Of Frost Bites'].index, inplace=True)

for i in strings:
    dfMoviesFinal[i] = dfMoviesFinal[i].astype("string")
for i in integers:
    dfMoviesFinal[i] = dfMoviesFinal[i].astype("int")
for i in floats:
    dfMoviesFinal[i].replace("", 0, inplace=True) #hay strings vacios en vote_count por eso los lleno con 0
    dfMoviesFinal[i] = dfMoviesFinal[i].astype("float")
dfMoviesFinal["release_date"] = pd.to_datetime(dfMoviesFinal["release_date"])

In [97]:
#uso feature extraction para trabajar con los resultados del EDA sobre wordclouds

#nube de palabras para la columna title
#palabras mas repetidas: love, day, man, girl

#keywords para que arme las columnas
keywordsTitle = ["day", "man", "girl"]

#origen del texto
text = dfMoviesFinal["movie_title"]

#inicializo la vectorizacion
vectorizer = CountVectorizer(vocabulary=keywordsTitle, lowercase=True)

#cuenta cuantas veces una keyword esta en un movie title
feature = vectorizer.transform(dfMoviesFinal["movie_title"])

#crea df que almacena cuantas veces se cuenta en el titulo de cada pelicula cada feature
feature_dfTitles = pd.DataFrame(feature.toarray(), columns=vectorizer.get_feature_names_out())

#concateno y dropeo la columna movie_title
dfMoviesFinal = pd.concat([feature_dfTitles, dfMoviesFinal], axis=1)
dfMoviesFinal.drop(columns=["movie_title"], inplace=True)


In [98]:
#nube de palabras para la columna overview
#palabras mas repetidas: life, find, love, one

#keywords para que arme las columnas
keywordsOverview = ["life", "find", "love", "one"]

#origen del texto
text = dfMoviesFinal["overview"]

#inicializo la vectorizacion
vectorizer = CountVectorizer(vocabulary=keywordsOverview, lowercase=True)

#cuenta cuantas veces una keyword esta en un movie title
feature = vectorizer.transform(dfMoviesFinal["overview"])

#crea df que almacena cuantas veces se cuenta en el titulo de cada pelicula cada feature
feature_dfOverview = pd.DataFrame(feature.toarray(), columns=vectorizer.get_feature_names_out())

#concateno y dropeo la columna movie_title
dfMoviesFinal = pd.concat([feature_dfOverview, dfMoviesFinal], axis=1)
dfMoviesFinal.drop(columns=["overview"], inplace=True)
dfMoviesFinal

,life,find,love,one,day,man,girl,budget,movie_genre,franquicia,...,release_year,revenue,runtime,language,vote_average,vote_count,return,director,character,actor
0,0,0,0,0,0,0,0,30000000.0,Animation,Toy Story Collection,...,1995,373554033.0,81.0,English,7.7,5415,12.45,John Lasseter,Woody (voice),Tom Hanks
1,0,1,0,0,0,0,0,65000000.0,Adventure,0,...,1995,262797249.0,104.0,English,6.9,2413,4.04,Joe Johnston,Alan Parrish,Robin Williams
2,0,0,0,0,0,0,0,0.0,Romance,Grumpy Old Men Collection,...,1995,0.0,101.0,English,6.5,92,0.00,Howard Deutch,Max Goldman,Walter Matthau
3,0,1,0,0,0,0,0,16000000.0,Comedy,0,...,1995,81452156.0,127.0,English,6.1,34,5.09,Forest Whitaker,Savannah 'Vannah' Jackson,Whitney Houston
4,0,0,0,0,0,0,0,0.0,Comedy,Father of the Bride Collection,...,1995,76578911.0,106.0,English,5.7,173,0.00,Charles Shyer,George Banks,Steve Martin
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50068,0,0,0,0,0,0,0,0.0,Genero Desconocido,0,...,1991,0.0,104.0,0,5.7,26,0.00,John Irvin,Sir Robert Hode,Patrick Bergin
50069,0,0,0,0,0,0,0,0.0,Genero Desconocido,0,...,2011,0.0,360.0,0,9.0,3,0.00,Lav Diaz,Sister Angela,Angel Aquino
50070,0,0,0,1,0,0,0,0.0,Genero Desconocido,0,...,2003,0.0,90.0,0,3.8,6,0.00,Mark L. Lester,Emily Shaw,Erika Eleniak
50071,1,0,0,3,0,0,0,0.0,Genero Desconocido,0,...,1917,0.0,87.0,0,0.0,0,0.00,Yakov Protazanov,,Iwan Mosschuchin


In [99]:
dfMoviesFinal[dfMoviesFinal["love"]>0].count()

life                  4703
find                  4703
love                  4703
one                   4703
day                   4703
man                   4703
girl                  4703
budget                4703
movie_genre           4703
franquicia            4703
original_language     4703
popularity            4703
production_company    4703
production_country    4703
release_date          4703
release_year          4703
revenue               4703
runtime               4703
language              4703
vote_average          4703
vote_count            4703
return                4703
director              4703
character             4703
actor                 4703
dtype: int64

In [ ]:
#Binary Encoding:
#Hashing Encoding:
#Target encoding
#label encoding

largueUniqueCountCols = ["movie_title", "director", "character", "actor", "production_company"]

otherCols = ["franquicia", "original_language"]

dfMoviesFinal["production_company"].nunique()